In [1]:
!pip install python-anilist

In [133]:
import json
import time
import re
import sys

In [28]:
data = json.load(open("anime-offline-database.json", "r", encoding="utf-8"))

def anime_filter(anime):

    if not anime["animeSeason"]["year"]:
        return False
    
    if anime["animeSeason"]["year"] <= 2014:
        return False
    
    if anime["type"] != "TV":
        return False

    if anime["status"] != "FINISHED":
        return False

    anilist_id = list(filter(lambda x: "anilist.co" in x, anime["sources"]))
    if len(anilist_id) == 0:
        return False

    return True

anime_filtered = list(filter(anime_filter, data["data"]))
len(anime_filtered)

2066

In [107]:
json.dump(anime_filtered, open("anime-filtered.json", "w+"))
anime_filtered = json.load(open("anime-filtered.json", "r"))

In [30]:
def anime_map(anime):
    anilist_url = list(filter(lambda x: "anilist.co" in x, anime["sources"]))[0]
    anilist_id = int(re.search("https://anilist.co/anime/(\d*)", anilist_url).group(1))

    return {
        "title": anime["title"],
        "anilist_id": anilist_id,
        "year": anime["animeSeason"]["year"], 
        "season": anime["animeSeason"]["season"],
    }

anime_mapped = list(map(anime_map, anime_filtered))

In [109]:
json.dump(anime_mapped, open("anime-mapped.json", "w+"))
anime_mapped = json.load(open("anime-mapped.json", "r"))

In [ ]:
characters = {}

In [164]:
from anilist import Client, types, utils
from anilist.types import Anime, Ranking
from anilist.utils import ANIME_GET_QUERY, API_URL, HEADERS
import httpx

class CustomClient(Client):
    def get_anime(self, id: int) -> Anime | None:
        if not self.httpx:
            self.httpx = httpx.Client()
        response = self.httpx.post(
            url=API_URL,
            json=dict(
                query=ANIME_GET_QUERY,
                variables=dict(
                    id=id,
                    MediaType="ANIME",
                ),
            ),
            headers=HEADERS,
        )
        # if response.status_code != 200:
        print(response.headers)
        print(response.text)
        data = response.json()
        if data["data"]:
            try:
                item = data["data"]["Page"]["media"][0]
                return Anime(
                    id=item["id"],
                    title=item["title"],
                    url=item["siteUrl"],
                    episodes=item["episodes"],
                    description=item["description"],
                    format=item["format"],
                    status=item["status"],
                    duration=item["duration"],
                    genres=item["genres"],
                    is_adult=item["isAdult"],
                    tags=item["tags"],
                    studios=item["studios"],
                    start_date=item["startDate"],
                    end_date=item["endDate"],
                    season=dict(
                        name=item["season"],
                        year=item["seasonYear"],
                        number=item["seasonInt"],
                    ),
                    country=item["countryOfOrigin"],
                    cover=item["coverImage"],
                    banner=item["bannerImage"],
                    source=item["source"],
                    hashtag=item["hashtag"],
                    synonyms=item["synonyms"],
                    score=dict(
                        mean=item["meanScore"],
                        average=item["averageScore"],
                    ),
                    next_airing=item["nextAiringEpisode"],
                    trailer=item["trailer"],
                    staff=item["staff"],
                    characters=item["characters"],
                    popularity=item["popularity"],
                    rankings=[
                        Ranking(
                            type=i["type"],
                            all_time=i["allTime"],
                            format=i["format"],
                            rank=i["rank"],
                            year=i["year"],
                            season=i["season"],
                        )
                        for i in item["rankings"]
                    ],
                    relations=item["relations"],
                )
            except Exception:
                pass
        return None


client = CustomClient()
# ignorelist = []

for i, anime in enumerate(anime_mapped):
    sys.stdout.write(f"\r[ {i} / {len(anime_mapped)} ]")

    if anime["anilist_id"] in characters.keys() or anime["anilist_id"] in ignorelist:
        continue

    res = client.get_anime(anime["anilist_id"])

    if res is None:
        break
        continue

    res = res.raw()

    if not "characters" in res:
        ignorelist.append(anime["anilist_id"])
        continue

    anime_characters = list(res["characters"])
    anime_characters = list(map(lambda x: x.raw(), anime_characters))
    
    characters[anime["anilist_id"]] = anime_characters

    time.sleep(3)


[ 2057 / 2066 ]Headers({'date': 'Thu, 16 May 2024 19:38:20 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'cache-control': 'no-cache, private', 'x-ratelimit-limit': '90', 'x-ratelimit-remaining': '89', 'set-cookie': 'laravel_session=9XLCUln5TBUH6MgmuKYUiT9ANin4xErZYurNTj4u; expires=Fri, 17-May-2024 07:38:20 GMT; Max-Age=43200; path=/; httponly', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'x-content-type-options': 'nosniff', 'access-control-allow-origin': '*', 'access-control-allow-methods': 'GET, POST, OPTIONS', 'access-control-allow-headers': 'Authorization,Accept,DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Content-Range,Range', 'access-control-expose-headers': 'X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, Content-Length, Content-Range', 'cf-cache-status': 'DYNAMIC', 'report-to': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare

In [165]:
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, types.Anime):
            return obj.raw()
        if isinstance(obj, types.Name):
            return obj.raw()
        if isinstance(obj, types.Character):
            return obj.raw()
        return super().default(obj)

json.dump(characters, open("characters.json", "w+"), cls=CustomEncoder)

In [166]:
characters = json.load(open("characters.json", "r"))
keys = list(characters.keys())

for c in keys:
    characters[int(c)] = characters[c]
    del characters[c]